In [34]:
import requests
from pyquery import PyQuery as pq
from tqdm import tqdm

import datetime
import os

In [32]:
startpage = "https://brechner.org/resources/brechner-report/"
recentpage = "https://us9.campaign-archive.com/home/?u=e2bfc368fca7a023b36d90db8&id=91f7a1708d"
rawdir = "archives/"
filepre = "Brechner_"
filepost = ".html"

for localdir in [rawdir]:
    os.makedirs(localdir, exist_ok=True)

In [7]:
r = requests.get(startpage)

In [18]:
html = r.text
oldblock = html.split("<h2>By Year</h2>")[1].strip().split("</div>")[0]

In [19]:
yeargroups = oldblock.split("<h3>")

In [44]:
allurls = []

In [45]:
for yeargroup in tqdm(yeargroups):
    if yeargroup:
        myyear = yeargroup.split("</h3>")[0]
        for aholder in pq(yeargroup)("a"):
            remoteurl = pq(aholder)("a").attr("href")
            allurls.append(remoteurl)
            monthname = pq(aholder)("a").text().split("/")[0]    # Fix for "June/July 2016"
            if "Annual" not in monthname:
                realdate = datetime.datetime.strptime(f"{monthname} {myyear}", "%B %Y")
                mydate = realdate.strftime("%Y-%m-%d")
                targetfilename = rawdir + filepre + mydate + filepost
                if not os.path.exists(targetfilename):
                    r = requests.get(remoteurl)
                    if r.status_code != 200:
                        print(f"Error downloading file for {mydate} from {remoteurl}")
                    else:
                        with open(targetfilename, "wb") as outfile:
                            outfile.write(r.content)

100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 402.13it/s]


In [37]:
r = requests.get(recentpage)
html = r.text

In [38]:
archivelist = pq(html)("ul#archive-list")

In [46]:
for entry in tqdm(pq(archivelist)("li")):
    rawdate = pq(entry)("li").text().strip().split()[0]
    gooddate = datetime.datetime.strptime(rawdate, "%m/%d/%Y")
    mydate = gooddate.strftime("%Y-%m-%d")
    targetfilename = rawdir + filepre + mydate + filepost
    remoteurl = pq(entry)("a").attr("href")
    allurls.append(remoteurl)
    if not os.path.exists(targetfilename):
        r = requests.get(remoteurl)
        if r.status_code != 200:
            print(f"Error downloading file for {mydate} from {remoteurl}")
        else:
            with open(targetfilename, "wb") as outfile:
                outfile.write(r.content)    

100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 4011.19it/s]
